## Wandb에서 pt 파일을 받아서, ONNX Export 후에 다시 업로드

### Wandb 설치

In [ ]:
# Install the Weights & Biases library
!pip install wandb -qqq

In [ ]:
# Log a dataset version as an artifact
import wandb
import os

In [ ]:
wandb.login()

### Latest 모델 다운로드

In [ ]:
run = wandb.init()
artifact = run.use_artifact('avikus-cv/model-registry/neuboat:pt', type='model')
artifact_dir = artifact.download()

In [ ]:
model_path = './artifacts/neuboat:latest/best.pt'

### Export onnx

In [ ]:
command = f"python ../export_yoloV5_for_deepstream.py -w {model_path} -s 736 1280 --opset 12 --simplify --dynamic"
!{command}

In [ ]:
import os

current_filename = 'best.onnx'
new_filename = 'neuboat.onnx'

os.rename(current_filename, new_filename)

In [ ]:
labels_file = 'labels.txt'
labels = []

with open(labels_file, 'r') as file:
    for line in file:
        label = line.strip()  # Remove leading/trailing whitespaces
        labels.append(label)

print(labels)

In [ ]:
art = wandb.Artifact(f"nature-{wandb.run.id}", 
                        type="model",
                        metadata={'format': 'onnx',
                                  'model_input_size': '736 1280',
                                  'classes': labels})

art.add_file(new_filename)

### Add aliases to keep track of your best checkpoints over time
wandb.log_artifact(art, aliases=["best", "latest"])

In [ ]:
run.link_artifact(art, 'avikus-cv/model-registry/neuboat', aliases=['best', 'onnx'])

In [ ]:
wandb.finish()